In [26]:
# 그래프 및 tf.function


# 그래프란?
# 그래프 실행은 Python 외부에서 이식성을 가능하게 하며 성능이 더 우수한 경향이 잇음
# 그래프 실행은 텐서 계산이 tf.Graph 또는 간단히 '그래프'라고도 하는 TensorFlow 그래프로 실행됨을 의미
# 그래프는 계산의 단위를 나타내는 tf.Operation 객체와 연산 간 흐르는 데이터의 단위를 나타내는 tf.Tensor 객체의 세트를 포함
# 데이터 구조는 tf.Graph 컨텍스트에서 정의되며, 그래프는 데이터 구조이므로 원래 Python 코드 없이 저장, 실행 및 복원 가능

# 그래프의 이점
# 그래프를 사용하면 유연성이 크게 향상 (Python 인터프리터가 없는 환경에서 TensorFlow 그래프 사용 가능)
# TensorFlow는 그래프를 Python에서 내보낼 때 저장된 모델의 형식으로 그래프를 사용
# 그래프는 쉽게 최적화되어 컴파일러가 다음과 같은 변환을 수행할 수 있음
# 1. 계산에서 상수 노드를 접어 텐서의 값을 정적으로 추론('일정한 접기')
# 2. 독립적인 계산의 하위 부분을 분리해 스레드 또는 기기 간에 분할
# 3. 공동 하위 표현식을 제거해 산술 연산을 단순화
# 요약하면, 그래프는 TensorFlow가 빠르게, 병렬로, 그리고 효율적으로 여러 기기에서 실행할 때 매우 유용


# 그래프 이용
# tf.function을 직접 호출 또는 데코레이터로 사용하여 TensorFlow에서 그래프를 만들고 실행
# tf.function은 일반 함수를 입력으로 받아 Function을 반환
# Function은 Python 함수로부터 TensorFlow 그래프를 빌드하는 Python callable
import tensorflow as tf
import timeit
from datetime import datetime

# Define a Python function
# def a_regular_function(x, y, b):
#     x = tf.matmul(x, y)
#     x = x + b
#     return x

# 'a_function_that_uses_a_graph' is a TensorFlow 'Function'
# a_function_that_uses_a_graph = tf.function(a_regular_function)

# Make some tensors
# x1 = tf.constant([[1.0, 2.0]])
# y1 = tf.constant([[2.0], [3.0]])
# b1 = tf.constant(4.0)

# orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a 'Function' like a Python function
# tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
# assert(orig_value == tf_function_value)

# Function는 하나의 API 뒤에서 여러 tf.Graph를 캡슐화 (속도 및 배포 가능성과 같은 그래프 실행의 이점 제공 방식)
# def inner_function(x, y, b):
#     x = tf.matmul(x, y)
#     x = x + b
#     return x

# Use the decorator to make 'outer_function' a 'Function'
# @tf.function
# def outer_function(x):
#     y = tf.constant([[2.0], [3.0]])
#     b = tf.constant(4.0)
#     return inner_function(x, y, b)

# Note that the callable will create a graph that includes 'inner_funcion' as well as 'outer_function'
# outer_function(tf.constant([[1.0, 2.0]])).numpy()

## Python 함수를 그래프로 변환
# tf.function은 AutoGraph(tf.autograph) 라이브러리를 사용하여 Python 코드를 그래프 생성 코드로 변환
# def simple_relu(x):
#     if tf.greater(x, 0):
#         return x
#     else:
#         return 0

# 'tf.simple_relu' is a TensorFlow 'Function' that wraps 'simple_relu'
# tf_simple_relu = tf.function(simple_relu)
# print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
# print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

# This is the graph-generating output of AutoGraph
# print(tf.autograph.to_code(simple_relu))

# This is the graph itself
# print(tf_simple_relu.get_concrete_function(tf.constant(1)).graph.as_graph_def())

## 다형성 : 하나의 Function, 다수의 그래프
# 기존 그래프에서 처리할 수 없는 인수 세트로 Function을 호출할 때마다 Function은 이러한 새 인수에 특화된 새 tf.Graph를 생성
# tf.Graph 입력의 유형 사양을 입력 서명 또는 서명이라 함
# Function은 해당 서명에 대응하는 tf.Graph를 ConcreteFunction에 저장 (ConcreteFunction은 tf.Graph를 감싸는 래퍼)
# @tf.function
# def my_relu(x):
#     return tf.maximum(0., x)

# 'my_relu' creates new graphs as it observes more signatures
# print(my_relu(tf.constant(5.5)))
# print(my_relu([1, -1]))
# print(my_relu(tf.constant([3., -3.])))

# These two calls do not create new graphs
# print(my_relu(tf.constant(-2.5)))               # Signature matches 'tf.constant(5.5)'
# print(my_relu(tf.constant([-1., 1.])))          # Signature matches 'tf.constant([3., -3.])'

# 여러 그래프로 뒷받침된다는 점에서 Function은 다형성의 특징을 가짐
# 단일 tf.Graph로 나타낼 수 있는 것보다 더 많은 입력 유형을 지원하고 tf.Graph가 더 우수한 성능을 갖도록 최적화
# There are three 'ConcreteFunction's (one for each graph) in 'my_relu'
# The 'ConcreteFunction' also knows the return type ans shape
# print(my_relu.pretty_printed_concrete_signatures())


# tf.function 사용
## 그래프 실행 vs 즉시 실행
# Function의 코드는 즉시 실행 또는 그래프 실행이 가능
# 기본적으로 Function은 코드를 그래프로 실행
# @tf.function
# def get_MSE(y_true, y_pred):
#     sq_diff = tf.pow(y_true - y_pred, 2)
#     return tf.reduce_mean(sq_diff)

y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
# print(y_true)
# print(y_pred)

# get_MSE(y_true, y_pred)

# Function의 그래프가 동등한 Python 함수와 같은 계산을 하는지 확인하고자 tf.config.run_functions_eagerly(True)로 즉시 실행
# 코드를 정상적으로 실행하는 대신 그래프를 생성하고 실행하는 Function의 기능을 해제
# tf.config.run_functions_eagerly(True)

# get_MSE(y_true, y_pred)

# Don't forget to set it back when you are done
# tf.config.run_functions_eagerly(False)

# Function은 그래프 및 즉시 실행에서 서로 다르게 동작 가능
# @tf.function
# def get_MSE(y_true, y_pred):
#     print("Calculating MSE!")
#     sq_diff = tf.pow(y_true - y_pred, 2)
#     return tf.reduce_mean(sq_diff)

# print문은 Function이 원래 코드를 실행할 때 실행되며, 이 때 '트레이싱'이라는 프로세스를 통해 그래프 생성
# 추적은 TensorFlow 연산을 그래프로 캡쳐하고 print는 그래프로 캡쳐되지 않음
# error = get_MSE(y_true, y_pred)
# error = get_MSE(y_true, y_pred)
# error = get_MSE(y_true, y_pred)

# Now, globally set everything to run eagerly to force eager execution
# tf.config.run_functions_eagerly(True)

# error = get_MSE(y_true, y_pred)
# error = get_MSE(y_true, y_pred)
# error = get_MSE(y_true, y_pred)

# tf.config.run_functions_eagerly(False)

## 비평가(Non-strict) 실행
# 그래프 실행은 다음을 포함하여 관찰 가능한 효과를 생성하는 데 필요한 작업만 실행
# 1. 함수의 반환 값
# 2. 잘 알려진 부작용(tf.print와 같은 입/출력 작업, tf.debugging의 어설션 기능과 같은 디버깅 작업, tf.Variable의 변형)
# 특히, 런타임 오류 검사는 관찰 가능한 효과로 간주되지 않음
# def unused_return_eager(x):
#     # Get index 1 will fail when 'len(x) == 1'
#     tf.gather(x, [1])                           # unused
#     return x

# try:
#     print(unused_return_eager(tf.constant([0.0])))
# except tf.errors.InvalidArgumentError as e:
#     # All operations are run during eager execution so an error is raised
#     print(f"{type(e).__name__}: {e}")

# @tf.function
# def unused_return_graph(x):
#     tf.gather(x, [1])                           # unused
#     return x

# Only needed operations are run during graph execution. The error is not raised
# print(unused_return_graph(tf.constant([0.0])))

## tf.function 모범 사례
# 1. tf.config.run_functions_eagerly로 즉시 실행과 그래프 실행 사이를 조기에 자주 전환해 서로 (언제) 달라지는지 파악
# 2. Python 함수 외부에서 tf.Variable을 실행하고 수정은 내부에서 실행
# 3. tf.Variable과 Keras 객체를 제외하고 외부 Python 변수에 의존하는 함수를 작성하지 않아야 함
# 4. 텐서 및 기타 TensorFlow 유형을 입력으로 사용하는 함수를 작성하는 것이 좋음
# 5. 성능 이점을 극대화하기 위해 tf.function 하에서 계산이 가능한 많이 포함되도록 함


# 속도 향상 확인
# tf.function은 일반적으로 코드의 성능을 향상시키지만 속도 향상의 정도는 계산의 종류에 따라 다름
# 작은 계산의 경우 그래프를 호출하는 오버헤드에 의해 지배될 가능성
# tf.function은 일반적으로 훈련 루프의 속도를 높이는 데 사용
# x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

# def power(x, y):
#     result = tf.eye(10, dtype=tf.dtypes.int32)
#     for _ in range(y):
#         result = tf.matmul(x, result)
#     return result

# print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000), "seconds")

# power_as_graph = tf.function(power)
# print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000), "seconds")

## 성능과 상충 관계
# 그래프는 코드의 속도를 높일 수 있지만 이를 생성하는 프로세스에는 약간의 오베헤드가 존재
# 일부 함수의 경우 그래프를 생성하는 데 그래프를 실행하는 것보다 더 많은 시간이 소요
# 이 경우, 후속 실행에서 성능이 향상되는 보상이 빠르지만 대규모 모델 훈련의 첫 몇 단계에서는 트레이싱으로 인해 느려질 수 있음


# Function은 언제 트레이싱하는가?
# print문을 추가 (대략적인 규칙으로, Function은 트레이싱할 때마다 print문 실행)
@tf.function
def a_function_with_python_side_effect(x):
    print("Tracing!")                            # An eager-only side effect
    return x * x + tf.constant(2)

# This is traced the first time
# print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect
# print(a_function_with_python_side_effect(tf.constant(3)))

# This retraces each time the Python argument changes, as a Python argument could be an epoch count or other hyperparameter
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))

Tracing!
tf.Tensor(6, shape=(), dtype=int32)
Tracing!
tf.Tensor(11, shape=(), dtype=int32)
